# Tokenizer Examples

>A tokenizer is in charge of preparing the inputs for a model. The library contains tokenizers for all the models. Most of the tokenizers are available in two flavors: a full python implementation and a “Fast” implementation based on the Rust library 🤗 Tokenizers. The “Fast" implementations allows:
>
> * a significant speed-up in particular when doing batched tokenization and
> * additional methods to map between the original string (character and words) and the token space (e.g. getting the index of the token comprising a given character or the span of characters corresponding to a given token).
>
>The base classes [PreTrainedTokenizer](https://huggingface.co/docs/transformers/v4.39.0/en/main_classes/tokenizer#transformers.PreTrainedTokenizer) and [PreTrainedTokenizerFast](https://huggingface.co/docs/transformers/v4.39.0/en/main_classes/tokenizer#transformers.PreTrainedTokenizerFast) implement the common methods for encoding string inputs in model inputs (see below) and instantiating/saving python and “Fast” tokenizers either from a local file or directory or from a pretrained tokenizer provided by the library (downloaded from HuggingFace’s AWS S3 repository). They both rely on [PreTrainedTokenizerBase](https://huggingface.co/docs/transformers/v4.39.0/en/internal/tokenization_utils#transformers.PreTrainedTokenizerBase) that contains the common methods, and [SpecialTokensMixin](https://huggingface.co/docs/transformers/v4.39.0/en/internal/tokenization_utils#transformers.SpecialTokensMixin).
>
>[PreTrainedTokenizer](https://huggingface.co/docs/transformers/v4.39.0/en/main_classes/tokenizer#transformers.PreTrainedTokenizer) and [PreTrainedTokenizerFast](https://huggingface.co/docs/transformers/v4.39.0/en/main_classes/tokenizer#transformers.PreTrainedTokenizerFast) thus implement the main methods for using all the tokenizers:
>
> * Tokenizing (splitting strings in sub-word token strings), converting tokens strings to ids and back, and encoding/decoding (i.e., tokenizing and converting to integers).
> * Adding new tokens to the vocabulary in a way that is independent of the underlying structure (BPE, SentencePiece…).
> *Managing special tokens (like mask, beginning-of-sentence, etc.): adding them, assigning them to attributes in the tokenizer for easy access and making sure they are not split during tokenization.

Source: [https://huggingface.co/docs/transformers/main_classes/tokenizer](https://huggingface.co/docs/transformers/main_classes/tokenizer)

## Special Tokens

### Tokens [CLS] and [SEP]

BERT has been trained using the format: [CLS] sentence 1 [SEP] sentence 2 [SEP].

In [1]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

In [2]:
# Create a tokenizer that is compatible with the model "bert-base-uncased"
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [3]:
# Print out information regarding the tokenizer
tokenizer

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

### Convert Tokens to IDS and Back to Tokens

In [4]:
# Tokenize the string "hello world" and print the tokens
tokens = tokenizer.tokenize("hello world")
print(tokens)

['hello', 'world']


In [5]:
# Convert tokens to indices and print the indices
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[7592, 2088]


In [6]:
# Convert the indices back to tokens
tokenizer.convert_ids_to_tokens(ids)

['hello', 'world']

In [25]:
# Decode the indices, i.e. convert the ids back to a string
tokenizer.decode(ids)

'[CLS] hello world [SEP]'

In [46]:
# How are suffixes treated by this tokenizer?
# How are the words "go" and "going" converted into ids?
ids1 = tokenizer.convert_tokens_to_ids(tokenizer.tokenize("I will go"))
print(ids1)

ids2 = tokenizer.convert_tokens_to_ids(tokenizer.tokenize("I am going"))
print(ids2)

[1045, 2097, 2175]
[1045, 2572, 2183]


### Encode and Decode

In [26]:
# Encode the string. Tokenizes the string and converts it to indices and adds start, end and other tokens
ids = tokenizer.encode("hello world")
print(ids)

[101, 7592, 2088, 102]


In [27]:
tokenizer.decode(ids)

'[CLS] hello world [SEP]'

### Generate Input for The Model

In [28]:
model_inputs = tokenizer("hello world")
print(model_inputs)

{'input_ids': [101, 7592, 2088, 102], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}


In [29]:
text_input = ["I like cats.", "Do you like cats too?"]
tokenizer(text_input)

{'input_ids': [[101, 1045, 2066, 8870, 1012, 102], [101, 2079, 2017, 2066, 8870, 2205, 1029, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}

## Sequence Classification Example

When we create the sequence classification model, we get a warning saying that the head of the model has not been trained, i.e. the results we get are meaningless that this stage. We need to train the head in order to get meaningful results.

In [30]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
model_inputs = tokenizer("hello world", return_tensors='pt')

In [32]:
outputs = model(**model_inputs)
print(outputs)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.3159, -0.0049]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [33]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
outputs = model(**model_inputs)
print(outputs)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.3323,  0.3484, -0.2075]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [35]:
outputs.logits

tensor([[-0.3323,  0.3484, -0.2075]], grad_fn=<AddmmBackward0>)

In [36]:
outputs.logits.detach().cpu().numpy()

array([[-0.33225596,  0.34839675, -0.20746157]], dtype=float32)

In [37]:
# For batched inputs, depending on the version of 'transformers' library you are using, you might need to add the
# parameters padding=True and truncation=True if the output is in a tensor format. Tensors need to have same length.
text_input = ["I like cats.", "Do you like cats too?"]
model_inputs = tokenizer(text_input, return_tensors='pt', padding=True, truncation=True)

In [38]:
# print the model_inputs
print(model_inputs)

{'input_ids': tensor([[ 101, 1045, 2066, 8870, 1012,  102,    0,    0],
        [ 101, 2079, 2017, 2066, 8870, 2205, 1029,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1]])}


In [39]:
# print the input_ids
print(model_inputs.input_ids)

tensor([[ 101, 1045, 2066, 8870, 1012,  102,    0,    0],
        [ 101, 2079, 2017, 2066, 8870, 2205, 1029,  102]])


In [40]:
# print the attention mask, i.e. which tokens are used
print(model_inputs.attention_mask)

tensor([[1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1]])


In [41]:
# Do inference using the batch model_input
outputs = model(**model_inputs)

In [42]:
print(outputs)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.3043,  0.4186, -0.2840],
        [-0.3122,  0.4685, -0.3278]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
